# Shor's Algorithm for Factoring a Semiprime

This notebook implements a basic version of Shor's algorithm to factorize a semiprime number. In this example we use:

- **N = 15** (which factors as 3 × 5)
- **a = 7**

The notebook uses the `QuantumRingsLib` library for setting up and running the quantum circuit.

In [1]:
from dotenv import load_dotenv
import os
import math
from fractions import Fraction
import numpy as np
from matplotlib import pyplot as plt

# Import necessary components from QuantumRingsLib
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider, job_monitor, JobStatus

# Load environment variables
load_dotenv()
token = os.getenv("QUANTUM_RINGS_TOKEN")
name = os.getenv("QUANTUM_RINGS_NAME")

# Set up the provider and backend
provider = QuantumRingsProvider(token=token, name=name)
backend = provider.get_backend("scarlet_quantum_rings")
shots = 1024
provider.active_account()

{'name': 'tomoya-hatanaka@g.ecc.u-tokyo.ac.jp',
 'token': 'rings-200.A0bFllxTCX8Re3eWXHtkzVUc1B6xwhay',
 'max_qubits': '200'}

In [2]:
def inverse_qft(qc, qreg, n):
    """
    Applies the Inverse Quantum Fourier Transform (IQFT) to the first n qubits of qreg.
    
    Args:
        qc (QuantumCircuit): The quantum circuit.
        qreg (QuantumRegister): The quantum register on which the IQFT is applied.
        n (int): The number of qubits to apply the IQFT.
    """
    for i in range(n):
        for j in range(i):
            qc.cu1(-math.pi / float(2 ** (i - j)), qreg[i], qreg[j])
        qc.h(qreg[i])
    qc.barrier()
    return

In [3]:
def modular_multiplication(qc, control, target, a, N):
    """
    A placeholder implementation for a controlled modular multiplication gate.
    When the control qubit is |1>, this gate performs:
    
         |y⟩ -> |(a * y) mod N⟩
    
    For general a and N this operation is quite complex. Here, we provide a simple example
    for the case of N = 15 and a = 7. In practice, you would need a robust implementation.
    
    Args:
        qc (QuantumCircuit): The quantum circuit.
        control (Qubit): The control qubit.
        target (QuantumRegister): The target register.
        a (int): The multiplier.
        N (int): The modulus.
    """
    if N == 15 and a == 7:
        # Example: Use some CNOT and Toffoli (ccx) gates to simulate modular multiplication.
        qc.cx(target[1], target[2])
        qc.ccx(target[0], target[2], target[3])
    else:
        qc.barrier()
    return

In [4]:
def shors_algorithm(N, a):
    """
    Attempts to factorize the composite number N using Shor's algorithm with base a.
    
    Args:
        N (int): The composite number (e.g., semiprime) to factorize.
        a (int): An integer satisfying 1 < a < N and gcd(a, N) = 1.
    
    Returns:
        counts (dict): A dictionary of measurement outcomes.
    """
    # If a and N are not coprime, then a non-trivial factor has already been found.
    if math.gcd(a, N) != 1:
        return {f"non-trivial factor: {math.gcd(a, N)}": shots}
    
    # Determine the number of qubits:
    #   - n: Number of qubits in the control register (must be large enough so that 2^n > N^2)
    #   - m: Number of qubits in the target register (enough to represent numbers up to N)
    n = 2 * math.ceil(math.log(N, 2))
    m = math.ceil(math.log(N, 2))
    
    # Create the registers.
    control = QuantumRegister(n, 'control')
    target  = QuantumRegister(m, 'target')
    classical = ClassicalRegister(n, 'classical')
    qc = QuantumCircuit(control, target, classical)
    
    # --- Initialization ---
    # Apply Hadamard gates to all control qubits to create a superposition.
    for i in range(n):
        qc.h(control[i])
    # Initialize the target register to |1>.
    qc.x(target[0])
    qc.barrier()
    
    # --- Controlled Modular Exponentiation ---
    # For each control qubit j, apply the controlled multiplication by a^(2^j) mod N.
    for j in range(n):
        exp = 2 ** j
        a_exp = pow(a, exp, N)  # Compute a^(2^j) mod N.
        modular_multiplication(qc, control[j], target, a_exp, N)
    qc.barrier()
    
    # --- Apply Inverse QFT ---
    inverse_qft(qc, control, n)
    qc.barrier()
    
    # --- Measurement ---
    for i in range(n):
        qc.measure(control[i], classical[i])
    
    # Execute the quantum circuit.
    job = backend.run(qc, shots=shots)
    job_monitor(job)
    result = job.result()
    counts = result.get_counts()
    return counts

In [5]:
# Run Shor's Algorithm for N = 15, a = 7
N = 15
a = 7
factors = shors_algorithm(N, a)
print("Measurement Results:", factors)

Job Queued
Job Done.
Ending Job Monitor
Measurement Results: {'00000000': 1024}
